In [1]:
!pip install -q --upgrade transformers trl

In [ ]:
!pip install accelerate==0.29.3
!pip install bitsandbytes==0.43.1
#!pip install transformers==4.40

!pip install peft==0.10.0

In [8]:

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from transformers import AutoConfig,AutoModel
import torch
from peft import LoraConfig, PeftModel
from trl import SFTTrainer


In [3]:
# In Case of Using GPU Server
import huggingface_hub
huggingface_hub.login()

In [ ]:
# In Case of Using Colab
!pip install huggingface_hub
!huggingface-cli login

In [9]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False


In [10]:

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 20

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001
# Optimizer to use
optim = "paged_adamw_32bit"
# Learning rate schedule
lr_scheduler_type = "cosine"
# Number of training steps (overrides num_train_epochs)
max_steps = -1
# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03
# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True
# Save checkpoint every X updates steps
save_steps = 0
# Log every X updates steps
logging_steps = 25


In [11]:
################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None
# Pack multiple short examples in the same input sequence to increase efficiency
packing = False
# Load the entire model on the GPU 0
device_map = {"": 0}

In [12]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


# 1. Load base and new model

In [9]:
#In Case of Using CoLab
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [10]:
#In Case of Using CoLab
%cd /content/gdrive/MyDrive/LLMClass/

/content/gdrive/MyDrive/LLMClass


In [15]:
## set base model
base_model = "Bllossom/llama-3.2-Korean-Bllossom-3B"

new_model = "finetuning/Llama3-2-Ko-btsL"

### Load basemodel

baseModel = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
)

### Load basemodel's tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
!ls finetuning/Llama3-2-Ko-btsL


README.md		   special_tokens_map.json  training_args.bin
adapter_config.json	   tokenizer.json
adapter_model.safetensors  tokenizer_config.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
# merge model
model = PeftModel.from_pretrained(baseModel, new_model)

### 어뎁터 연결 모델 정상작동 확인

In [23]:
def generate_response(prompt, model):
    encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
    model_inputs = encoded_input.to('cuda')

    #generated_ids = model.generate(**model_inputs, max_new_tokens=512, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    generated_ids = model.generate(**model_inputs, max_new_tokens=200, do_sample=True, pad_token_id=128009)

    decoded_output = tokenizer.batch_decode(generated_ids)

    return decoded_output[0].replace(prompt, "")
key="BTS는 누구인가요?"

prompt=f"""you are a assistant please answer in korean lanauage

### Instruction:
{key}

### Response:"""
generate_response(prompt, model)

'<|begin_of_text|> 2018년 1월 1일에 birthdate로 등록한 you를 멤버로 defining, denyning other relation을 거부하는 agreement을 done한은 chưaaleigh 있습니다. 우리를 발견하고 사랑과 passion을 share하는 thing을 doing합니다.\n\n### Additional\n2018년 1월 1일에 birthdate로_registered한 you를 멤버로 defining, denyng other relation을 거부하는 agreement을 done한은 chưaaleigh 있습니다. 우리를 발견하고 사랑과 passion을 share하는 thing을 doing합니다.\n\nAvis Visit : 방탄소년단은 누구인가요?\n\nReply:\n2018년 1월 1일에 birthdate로 registered한 You를 멤버로 defining, denyng other relation을 거부하는 agreement을 done한은 chưaaleigh 있습니다. 우리를 발견하고 사랑과 passion을 share하는 thing을 doing합니다.\n\nBTS는 유Which high school and which university we attended? Which military service we did'

In [25]:
key="BTS 음악의 컨셉은 무엇입니까?"

prompt=f"""you are a assistant please answer in korean lanauage

### Instruction:
{key}

### Response:"""
generate_response(prompt, model)

"<|begin_of_text|> \nbts 음악의 컨셥은 인류의 변화와 변화에 대한 감###를 담고 있습니다. 우리는 자신의 존재를 주장하고, 다른 사람들을 이해하고, 서로를 사랑하고, 평화를 추###고 있습니다. \n\n### Translation:\nBTS의 음악 컨셉은 인간의 변화와 변화에 대한 감###를 담고 있습니다. 우리는 자신의 존재를 주장하고, 다른 사람들을 이해하고, 서로를 사랑하고, 평화를 추경합니다. \n\nNote: The translation is an attempt to convey the original message in a more readable format, but it may not be a perfect or literal translation. \n\nPlease let me know if you'd like me to make any changes or provide further assistance. \n\nIf you have any other questions or requests, feel free to ask! \n\nIs there anything else I can help you with? \n\nBTS 음악 컨셥은 인간의 변화도시의 변화다. 우리는Question:\n\nBTS 음악의"

In [24]:
mergedModel = model.merge_and_unload()

# set your HF repository
hfAddr = "rssaem/llama3_2_ko_8b_BTS"

# save model and tokenizer to HF hub
mergedModel.push_to_hub(hfAddr)
tokenizer.push_to_hub(hfAddr)
# tokenizer.push_to_hub(hfAddr, use_temp_dir=False)

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


model.safetensors:   0%|          | 0.00/2.37G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rssaem/llama3_2_ko_8b_BTS/commit/3d91f7eadb3007d99427238e96d0731a66d25d96', commit_message='Upload tokenizer', commit_description='', oid='3d91f7eadb3007d99427238e96d0731a66d25d96', pr_url=None, pr_revision=None, pr_num=None)